In [ ]:
import os

os.environ['GENSIM_DATA_DIR'] = './models'

In [ ]:
import gensim.downloader as api
import pandas as pd

info_df = pd.DataFrame.from_dict(api.info()['models'], orient='index')
info_df[['file_size', 'base_dataset', 'parameters']].head(5)

In [ ]:
model = api.load("glove-wiki-gigaword-50")

In [ ]:
# Análise de similaridade
v_king = model['king']
v_queen = model['queen']

print("Vector size:", model.vector_size)
print("v_king  =", v_king[:10])
print("v_queen =", v_queen[:10])
print("similarity:", model.similarity('king', 'queen'))

model.most_similar('king', topn=3)

v_lion = model['lion']
v_nano = model['nanotechnology']

model.cosine_similarities(v_king, [v_queen, v_lion, v_nano])
# Palavras mais similares fruto de uma subtração entre o somatório vetores
model.most_similar(positive=['woman', 'king'], negative=['man'], topn=3)
model.most_similar(positive=['paris', 'germany'], negative=['france'], topn=3)
# Palavra mais similar com base apenas em um somatório
model.most_similar(positive=['france', 'capital'], topn=1)


In [29]:
import sqlite3
import pandas as pd

posts_df = pd.read_csv("/home/higoreller/pos_projects/datasets/class_6/rspct.tsv", sep='\t')
subred_df = pd.read_csv("/home/higoreller/pos_projects/datasets/class_6/subreddit_info.csv").set_index(['subreddit'])
df = posts_df.join(subred_df, on='subreddit')
df.head(5)
print(df.columns)
column_mapping = {
    'id': 'id',
    'subreddit': 'subreddit',
    'title': 'title',
    'selftext': 'text',
    'category_1': 'category',
    'category_2': 'subcategory',
    'category_3': None, # no data
    'in_data': None, # not needed
    'reason_for_exclusion': None # not needed
}
# define remaining columns
columns = [c for c in column_mapping.keys() if column_mapping[c] != None]

# select and rename those columns
df = df[columns].rename(columns=column_mapping)
df = df[df['category'] == 'autos']
df.to_pickle("reddit_dataframe.pkl")

Index(['id', 'subreddit', 'title', 'selftext', 'category_1', 'category_2',
       'category_3', 'in_data', 'reason_for_exclusion'],
      dtype='object')


In [30]:
db_name = "/home/higoreller/pos_projects/datasets/class_6/reddit-selfposts.db"
con = sqlite3.connect(db_name)
df = pd.read_sql("select subreddit, lemmas, text from posts_nlp", con)
con.close()

df['lemmas'] = df['lemmas'].str.lower().str.split() # lower case tokens
sents = df['lemmas'] # our training "sentences"

DatabaseError: Execution failed on sql 'select subreddit, lemmas, text from posts_nlp': no such table: posts_nlp

In [ ]:
from gensim.models.phrases import Phrases, npmi_scorer

phrases = Phrases(sents, min_count=10, threshold=0.3, delimiter=b'-', scoring=npmi_scorer)

In [ ]:
sent = "I had to replace the timing belt in my mercedes c300".split()
phrased = phrases[sent]
print('|'.join(phrased))

"""I|had|to|replace|the|timing-belt|in|my|mercedes-c300"""

In [ ]:
phrase_df = pd.DataFrame(phrases.export_phrases(sents),
                         columns =['phrase', 'score'])
phrase_df = phrase_df[['phrase', 'score']].drop_duplicates() \
            .sort_values(by='score', ascending=False).reset_index(drop=True)
phrase_df['phrase'] = phrase_df['phrase'].map(lambda p: p.decode('utf-8'))

In [ ]:
phrase_df[phrase_df['phrase'].str.contains('mercedes')]

"""mercedes c300 => 0.47 / mercedez benz => 0.80"""